In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import re
import csv
import os 
import matplotlib.pyplot as plt
import random
import numpy as np
tf.reset_default_graph()


# path_train = "/home/manmiddle/GPU_WS/128x128_cd/128x128_cd_img_train_26000/"
# path_test = "/home/manmiddle/GPU_WS/128x128_cd/128x128_cd_img_test_4000/"
# label_csv_train = "/home/manmiddle/GPU_WS/128x128_cd/26000_label_linux.csv"
# label_csv_test = "/home/manmiddle/GPU_WS/128x128_cd/4000_label_linux.csv"

#croos validation
path_train = "C:\\Users\\USER-PC\\GPU_KWS\\GPU_WS\\128x128_cd\\128x128_cd_30000\\"

label_csv_train = "C:\\Users\\USER-PC\\GPU_KWS\\GPU_WS\\128x128_cd\\128x128_cd_30000.csv"


# path_train = "/home/manmiddle/GPU_WS/128x128_cd/128x128_cd_img_train_26000/"
# path_test = "/home/manmiddle/GPU_Ws/128x128_cd/128x128_cd_img_test_3400/"
# label_csv_train = "/home/manmiddle/GPU_WS/128x128_cd/26000_label_linux.csv"
# label_csv_test = "/home/manmiddle/GPU_WS/128x128_cd/128x128_cd_label_test_3400.csv"

random.seed(1234)
################################ img_argument
def img_rotate(img):
    argument = [cv2.ROTATE_180,cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE]
    res_narray = np.zeros(shape = img.shape)
    for i in range(img.shape[0]):
        rotate_val = random.randint(0,2)
        res_narray[i] = cv2.rotate(img[i], argument[rotate_val])
    # print(res_narray)
    return res_narray

def img_contrast(color_img):
    #print("img_contrast_color_img.shape",color_img.shape)
    res_narray = np.zeros(shape = color_img.shape)
    #print("img_contrast_res_narray.shape_be",res_narray.shape)
    for i in range(color_img.shape[0]):
        contrast_val = random.randint(0,50)
        plus_img = np.zeros(shape = color_img[i].shape, dtype = np.uint8) + contrast_val
        res_narray[i] = cv2.add(color_img[i], plus_img)
    #print("img_contrast_res_narray.shape_af",res_narray.shape)
    return res_narray

def img_flip(img):
    res_narray = np.zeros(shape = img.shape)
    for i in range(img.shape[0]):
        flip_val = random.randint(0,1)
        res_narray[i] = cv2.flip(img[i], flip_val)    
    return res_narray

def img_normal(img):
    res_narray = np.zeros(shape = img.shape)
    for i in range(img.shape[0]):
        res_narray[i] = img[i]
    return res_narray

############################# 데이터 로드
def image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for i in file_list:
        a = int(re.sub('[^0-9]','',i))
        file_name.append(a)
        file_name.sort()
    data = []
    for i in file_name:
        file = path + str(i) + '.jpg'
        data.append(file)
    image = []
    for j in data:
        img = cv2.imread(j)
        image.append(img)     
    return np.array(image),data

def label_load(label_csv_test):    
    file = list(csv.reader(open(label_csv_test)))
    label = np.array(np.squeeze(np.array(file),1).astype(int)-1)
    label = np.eye(2)[label]
    return label

def next_batch(data_list,label,data,idx,batch_size):
    batch1 = data_list[idx * batch_size:idx * batch_size + batch_size]
    label2 = label[idx * batch_size:idx * batch_size + batch_size]
    data3 = data[idx * batch_size:idx * batch_size + batch_size]
    index = list(np.arange(len(batch1)))
    random.shuffle(index)
    index = np.array(index)
    data = batch1[index]
    label = label2[index]
    path = []
    for i in index:
        path.append(data3[i])
    return data, label, path

def batch_norm_cnn(batch_image, depth) :
    epsilon = 1e-5
    beta = tf.Variable(tf.constant(0.0, shape=[depth]), trainable=True)
    gamma = tf.Variable(tf.constant(1.0, shape=[depth]), trainable=True)
    mean, variance = tf.nn.moments(batch_image, axes=[0, 1, 2])
    norm_batch = tf.nn.batch_normalization(batch_image, mean, variance, beta, gamma, epsilon)
    return norm_batch

def batch_norm_flat(batch_flat) :
    epsilon = 1e-5
    beta = tf.Variable(tf.constant(0.0, shape=[1]), trainable=True)
    gamma = tf.Variable(tf.constant(1.0, shape=[1]), trainable=True)
    mean, variance = tf.nn.moments(batch_flat, axes=[0])
    norm_batch = tf.nn.batch_normalization(batch_flat, mean, variance, beta, gamma, epsilon)
    return norm_batch

def shuffle_batch(data, label, path):
    index = np.arange(len(data))
    random.shuffle(index)
    res_data = data[index]
    res_label = label[index]
    res_path = []
    for i in index:
        res_path.append(path[i])
    return res_data, res_label, res_path


trainX,trainP = image_load(path_train)
print("trainX=",trainX.shape, "\ntrainP=",len(trainP))

trainY = label_load(label_csv_train)
print("trainY=",trainY.shape)

# trainX, trainY, trainP = shuffle_batch(trainX, trainY, trainP)

trainX1 = trainX[0:int(len(trainX)*0.2)]
trainX2 = trainX[int(len(trainX)*0.2):int(len(trainX)*0.4)]
trainX3 = trainX[int(len(trainX)*0.4):int(len(trainX)*0.6)]
trainX4 = trainX[int(len(trainX)*0.6):int(len(trainX)*0.8)]
trainX5 = trainX[int(len(trainX)*0.8):int(len(trainX))]

trainX2345 = np.append(np.append(np.append(trainX2,trainX3,axis=0), trainX4, axis=0), trainX5, axis=0)
trainX1345 = np.append(np.append(np.append(trainX1,trainX3,axis=0), trainX4, axis=0), trainX5, axis=0)
trainX1245 = np.append(np.append(np.append(trainX1,trainX2,axis=0), trainX4, axis=0), trainX5, axis=0)
trainX1235 = np.append(np.append(np.append(trainX1,trainX2,axis=0), trainX3, axis=0), trainX5, axis=0)
trainX1234= np.append(np.append(np.append(trainX1,trainX2,axis=0), trainX3, axis=0), trainX4, axis=0)

# print("trainX1=",trainX1.shape)
print("trainX1=",trainX1.shape,"trainX2=",trainX2.shape,"trainX3=",trainX3.shape,"trainX4=",trainX4.shape,"trainX5=",trainX5.shape)
print("trainX1=",trainX2345.shape,"trainX2=",trainX1345.shape,"trainX3=",trainX1245.shape,"trainX4=",trainX1235.shape,"trainX5=",trainX1234.shape)

trainP1 = trainP[0:int(len(trainP)*0.2)]
trainP2 = trainP[int(len(trainP)*0.2):int(len(trainP)*0.4)]
trainP3 = trainP[int(len(trainP)*0.4):int(len(trainP)*0.6)]
trainP4 = trainP[int(len(trainP)*0.6):int(len(trainP)*0.8)]
trainP5 = trainP[int(len(trainP)*0.8):int(len(trainP))]

trainP2345 = np.append(np.append(np.append(trainP2,trainP3,axis=0), trainP4, axis=0), trainP5, axis=0)
trainP1345 = np.append(np.append(np.append(trainP1,trainP3,axis=0), trainP4, axis=0), trainP5, axis=0)
trainP1245 = np.append(np.append(np.append(trainP1,trainP2,axis=0), trainP4, axis=0), trainP5, axis=0)
trainP1235 = np.append(np.append(np.append(trainP1,trainP2,axis=0), trainP3, axis=0), trainP5, axis=0)
trainP1234= np.append(np.append(np.append(trainP1,trainP2,axis=0), trainP3, axis=0), trainP4, axis=0)

trainY1 = trainY[0:int(len(trainY)*0.2)]
trainY2 = trainY[int(len(trainY)*0.2):int(len(trainY)*0.4)]
trainY3 = trainY[int(len(trainY)*0.4):int(len(trainY)*0.6)]
trainY4 = trainY[int(len(trainY)*0.6):int(len(trainY)*0.8)]
trainY5 = trainY[int(len(trainY)*0.8):int(len(trainY))]

trainY2345 = np.append(np.append(np.append(trainY2,trainY3,axis=0), trainY4, axis=0), trainY5, axis=0)
trainY1345 = np.append(np.append(np.append(trainY1,trainY3,axis=0), trainY4, axis=0), trainY5, axis=0)
trainY1245 = np.append(np.append(np.append(trainY1,trainY2,axis=0), trainY4, axis=0), trainY5, axis=0)
trainY1235 = np.append(np.append(np.append(trainY1,trainY2,axis=0), trainY3, axis=0), trainY5, axis=0)
trainY1234= np.append(np.append(np.append(trainY1,trainY2,axis=0), trainY3, axis=0), trainY4, axis=0)

print("trainX1=",trainY1.shape,"trainX2=",trainY2.shape,"trainX3=",trainY3.shape,"trainX4=",trainY4.shape,"trainX5=",trainY5.shape)
print("trainX1=",trainY2345.shape,"trainX2=",trainY1345.shape,"trainX3=",trainY1245.shape,"trainX4=",trainY1235.shape,"trainX5=",trainY1234.shape)


selector = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
keep_prob2 = tf.placeholder(tf.float32)

trainX = trainX2345
trainY = trainY2345
trainP = trainP2345

testX = trainX1
testY = trainY1
testP = trainP1


In [ ]:
import time

############################### 하이퍼 파라미터
lr = 0.0001
train_size = len(trainX)
print(trainX.shape)
print(train_size)
test_size = len(testX)
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

total_epoch = 80
train_total_batch = int(train_size/batch_size)
test_total_batch = int(test_size/batch_size)


################################# 가중치 밑 편향,입력
x = tf.placeholder(tf.float32, [None, 128, 128, 3])
y = tf.placeholder(tf.float32, [None,2])
w_1 = tf.get_variable(name='w_1', shape=[8*8*100, 200], initializer=tf.contrib.layers.variance_scaling_initializer())
w_2 = tf.get_variable(name='w_2', shape=[200, 2], initializer=tf.contrib.layers.variance_scaling_initializer())
b_1 = tf.Variable(tf.ones(200))
b_2 = tf.Variable(tf.ones(2))


w1 = tf.Variable(tf.random_normal([10, 10, 3, 30], stddev=0.01))
w2 = tf.Variable(tf.random_normal([10, 10, 30, 40], stddev=0.01))
w3 = tf.Variable(tf.random_normal([7, 7, 40, 50], stddev=0.01))
w4 = tf.Variable(tf.random_normal([5, 5, 50, 60], stddev=0.01))
w5 = tf.Variable(tf.random_normal([3, 3, 60, 100], stddev=0.01))
w6 = tf.Variable(tf.random_normal([3, 3, 100, 100], stddev=0.01))


################################ Layer
# conv 1층
L1 = tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.contrib.layers.batch_norm(L1, is_training=selector)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob2)

# conv 2층
L2 = tf.nn.conv2d(L1, w2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.contrib.layers.batch_norm(L2, is_training=selector)
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob2)

# conv 3층
L3 = tf.nn.conv2d(L2, w3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.contrib.layers.batch_norm(L3, is_training=selector)
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob2)

# conv 4층
L4 = tf.nn.conv2d(L3, w4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.contrib.layers.batch_norm(L4, is_training=selector)
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
L4 = tf.nn.dropout(L4, keep_prob2)

# conv 5층
L5 = tf.nn.conv2d(L4, w5, strides=[1, 1, 1, 1], padding='SAME')
L5 = tf.contrib.layers.batch_norm(L5, is_training=selector)
L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5, keep_prob2)

# conv 6층
L6 = tf.nn.conv2d(L5, w6, strides=[1, 1, 1, 1], padding='SAME')
L6 = tf.contrib.layers.batch_norm(L6, is_training=selector)
L6 = tf.nn.relu(L6)
L6 = tf.nn.dropout(L6, keep_prob2)

conv_out = tf.reshape(L6, [-1, 8*8*100 ])


#hidden 1층
result1 = tf.matmul(conv_out,w_1)+b_1
batch1 = tf.contrib.layers.batch_norm(result1, is_training=selector)
relu1 = tf.nn.relu(batch1)
relu1 = tf.nn.dropout(relu1, keep_prob)
result_out = tf.matmul(relu1, w_2)+b_2

#출력층
result_hat = tf.nn.softmax(result_out)
result_predict = tf.argmax(result_hat, 1)
result_label = tf.argmax(y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(result_predict, result_label),tf.float32))
correct_prediction = tf.equal(result_predict, result_label)


# ################################### loss 함수, 손실함수, 학습방법 
delta = 1e-7
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(result_hat+delta), axis=1)) # 크로스엔트로피 loss값
optimizer = tf.train.AdamOptimizer(learning_rate=lr) # 경사하강법

# cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result_out, labels=y))
# optimizer =  tf.train.AdamOptimizer(learning_rate=lr)



######################################배치정규화 베타감마 적용하게하는부분
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    # train = optimizer.minimize(loss)
    train_step = optimizer.minimize(cross_entropy_loss)

saver = tf.train.Saver(save_relative_paths=True)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print("train_total_batch = ", train_total_batch)
    print("test_total_batch = ", test_total_batch)
    
    #total_wrong = []
    wrong_index1_1 = []
    # total_wrong_cnt = []
    for epoch in range(total_epoch):
        train_cost = 0
        train_accuracy = 0
        test_accuracy = 0
        global learningrate
        
        t0 = time.time()
        
        # cost
        for i in range(train_total_batch):
            img_func_list = [img_rotate, img_contrast, img_flip, img_normal]
            rand = random.randint(0,3)
            batch_x_train, batch_y_train, batch_p_train = next_batch(trainX, trainY, trainP,i,batch_size)
            _, cost = sess.run([train_step, cross_entropy_loss], feed_dict={x:img_func_list[rand](batch_x_train), y : batch_y_train, selector:True, keep_prob:0.5, keep_prob2:0.5})
            train_cost += cost
        train_loss_list.append(train_cost/train_total_batch)
        
        #train
        for i in range(train_total_batch):
            batch_x_train, batch_y_train ,batch_p_train = next_batch(trainX, trainY, trainP, i, batch_size)
            train_acc = sess.run(accuracy , feed_dict={x:batch_x_train, y:batch_y_train, selector:False, keep_prob:1,  keep_prob2:1})
            train_accuracy += train_acc
        train_acc_list.append(train_accuracy/train_total_batch)
        
        #test    
        for i in range(test_total_batch):
            batch_x_test, batch_y_test ,batch_p_test = next_batch(testX, testY, testP, i, batch_size)
            test_acc = sess.run(accuracy, feed_dict={x:batch_x_test, y:batch_y_test, selector:False, keep_prob:1,  keep_prob2:1})
            test_accuracy += test_acc
            
             
            wrong_test = sess.run(correct_prediction,feed_dict={x:batch_x_test,y:batch_y_test,selector:False,  keep_prob:1,  keep_prob2:1})
            for j in range(len(wrong_test)):
                if epoch==total_epoch-1:
                    if wrong_test[j] == False:
                        wrong_index1_1.append(batch_p_test[j]) 
            #total_wrong.append(wrong_index)
        test_acc_list.append(test_accuracy/test_total_batch)
        
        t1 = time.time()        
        total_time = t1-t0
        
        print('epoch :', epoch + 1,
              ' #  total_cost :', round(train_cost/train_total_batch, 3), # 1 에폭의 평균 cost
              ' #  train_accuracy :', round(train_accuracy/train_total_batch, 3),
              ' #  test_accuracy :', round(test_accuracy/test_total_batch, 3), 
              ' # 1_epoch_time : ', total_time)

x = np.arange(total_epoch)
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()